# Crypto Challenge Set 2

## 9. Implement PKCS#7 padding

https://cryptopals.com/sets/2/challenges/9

In [1]:
def pkcs7(block,blocksize,pad=b'\x04'):
    return block+max(0,(blocksize-len(block)))*pad

In [2]:
block = b"YELLOW SUBMARINE"
pkcs7(block,20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## 10. Implement CBC mode

https://cryptopals.com/sets/2/challenges/10

https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_block_chaining_(CBC)

In [3]:
from Crypto.Cipher import AES

def fixedXOR(a,b):
    return bytes([an^bn for an,bn in zip(a,b)])

def AES_CBC_decrypt(cipher,key):
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    IV = bsize*b"\x00"
    blocks = [ cipher[i:i+bsize] for i in range(0,len(cipher),bsize) ]
    plaintext = ""
    for i in range(len(blocks)):
        # decrypt block with AES ECB mode
        dec = aes.decrypt(blocks[i])
        # XOR with IV or previous cipher block
        dec = fixedXOR(dec,IV) if i==0 else fixedXOR(dec,blocks[i-1])
        plaintext += "".join([chr(c) for c in dec])
    return plaintext

In [4]:
from binascii import a2b_base64

key = b"YELLOW SUBMARINE"

with open("input/10.txt") as f:
    cipher = a2b_base64(f.read().replace("\n",""))
    plaintext = AES_CBC_decrypt(cipher,key)
    print(plaintext[0:241])
    
with open('input/plaintext.txt', 'w') as f:
    f.write(plaintext)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy


In [5]:
import binascii

def AES_CBC_encrypt(plaintext,key,IV=None):
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    if IV==None:
        IV = bsize*b"\x00"
    plainb = bytes(plaintext.encode()) # convert plaintext to bytes
    if len(plainb)%bsize: # padding to multiple of block size if needed 
        plainb = pkcs7(plainb,len(plainb)+bsize-len(plainb)%bsize)
    blocks = [ plainb[i:i+bsize] for i in range(0,len(plainb),bsize) ]
    ciphbl = []
    cipher = b""
    for i in range(len(blocks)):
        # XOR with IV or previous cipher block
        b = fixedXOR(blocks[i],IV) if i==0 else fixedXOR(blocks[i],ciphbl[i-1])
        # encrypt IVed block with AES ECB mode
        enc = aes.encrypt(b)
        ciphbl.append(enc)
        cipher += enc
    return cipher

In [6]:
cipher = AES_CBC_encrypt(plaintext,key)
plaintext = AES_CBC_decrypt(cipher,key)
print(plaintext[0:241])

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy


## 11. An ECB/CBC detection oracle

https://cryptopals.com/sets/2/challenges/11

In [7]:
# Write a function to generate a random AES key; that's just 16 random bytes.
# os.urandom() return a string of size random bytes suitable for cryptographic use.
# https://docs.python.org/3.8/library/os.html#os.urandom

import os

def randAESkey(keylen=16):
    return os.urandom(keylen)

randAESkey()

b'\x0c\xbd\xc0P\x1a\xe6Y9\xe4\x0bdW\xfd\xd5\xb2\x84'

In [8]:
# Write a function that encrypts data under an unknown key, e.g. a function that generates a random key 
# and encrypts under it. Under the hood, have the function append 5-10 bytes (count chosen randomly) 
# before the plaintext and 5-10 bytes after the plaintext. Have the function choose to encrypt under 
# ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). 

import os
import random
from Crypto.Cipher import AES

def encryption_oracle(plaintext):
    # generate a 16-bytes random key
    keysize = 16
    key = os.urandom(keysize)
    plainb = bytes(plaintext.encode()) # plaintext in bytes
    # prepend and append bytes
    plainb = os.urandom(random.randint(5,10))+plainb+os.urandom(random.randint(5,10)) 
    # pad the plaintext to a multiple of keysize
    if len(plainb)%keysize:
        plainb = pkcs7(plainb,len(plainb)+keysize-len(plainb)%keysize)
    cipher = ""
    mode = random.randint(0,1)
    if mode==1: # encrypt under ECB
        aes_ecb = AES.new(key, AES.MODE_ECB)
        cipher = aes_ecb.encrypt(plainb)
    else: # encrypt under CBC
        IV = os.urandom(keysize)
        aes_cbc = AES.new(key, AES.MODE_CBC, IV) 
        cipher = aes_cbc.encrypt(plainb)
    return mode, cipher

In [9]:
mode, cipher = encryption_oracle(plaintext)

In [10]:
# Detect the block cipher mode the function is using each time. 

def isAESECB(cipher,blocksize=16):
    blocks = [ cipher[i:i+blocksize] for i in range(0,len(cipher),blocksize) ]
    return bool(len(blocks) - len(set(blocks)))

In [11]:
ntot = 10
necb = 0

# This is the poem provided as ciphertext at challenge 10. No problem in guessing the cipher mode with it!
with open("input/plaintext.txt") as f:
    plaintext = f.read().strip("\n")

# This plaintext is too short and with no repetitions, it's impossible to detect ECB mode! 
#plaintext = "the quick brown fox jumps over the lazy dog"

# This is an abtract of an ATLAS paper (a real life text!): 
# it's relatively long, with some repetitions (but not many). Detection does not work :-(
#with open("input/HIGG-2018-51_abstract.txt") as f:
#    plaintext = f.read().strip("\n")

#print(plaintext)

for _ in range(ntot):
    mode, cipher = encryption_oracle(plaintext) # mode is the true encryption used (0 = CBC, 1 = ECB)
    pred = isAESECB(cipher)
    necb += pred
    print(mode,pred)

print(100*necb/ntot)

1 True
1 True
1 True
0 False
1 True
1 True
1 True
0 False
0 False
0 False
60.0


## 12. Byte-at-a-time ECB decryption (Simple)

https://cryptopals.com/sets/2/challenges/12

In [41]:
import binascii
import os
import random
from Crypto.Cipher import AES

def AES_ECB(mystring,unknown_str,key):
    plainb = (mystring+unknown_str).encode()
    # pad plaintext to multiple of keysize if needed
    keysize = len(key)
    if len(plainb)%keysize: 
        plainb = pkcs7(plainb,len(plainb)+keysize-len(plainb)%keysize)
    # encode
    aes_ecb = AES.new(key,AES.MODE_ECB)
    return aes_ecb.encrypt(plainb)

In [45]:
unknown = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
unknown_bin = binascii.a2b_base64(unknown)
unknown_str = unknown_bin.decode()

keysize = 16
key = os.urandom(keysize) # will use this (unknown) key for all the attacks

AES_ECB("",unknown_str,key)

b'E\xca*\xef\xbfL\xeb\x1d\xd7(\xfa\x0c)\x18\xe9\xed\xc5W\x00\xa9\x1dA\xd7\x8c\r\xb0\xeb\xf5\xe7\x7f\x1c\xf8\r#\x06\xac\xae,\xe8\x7f\xa7\xb8\xab\xdcB\xc85\xc2\xf1\xf7\x05\xe0Q\x18C\x19\x8e\x01a3\xb2\xe2@\x8d\x17\xe7)n\xff\x17\x89\x8bVq\x8dh\xff\xd0\x86\x03\x1c\xac>\x17\xdf\x88i\x18l\xb3H\r\x1c\xd0<\x04\xe7\xb4\xdf\x00\xa5\xf2\xf0%v\x94\x98\xa6m\xc9V,!ON\xa2p\xf3\x05;\xb0K\x7f\xb4\xc1\xc8\xf4\x0bH/t\xe1\x94Fw\xf48\xa6\xf1^7\x01/]'

In [47]:
# Discover the block size of the cipher
my_string = ""
cipher0 = AES_128_ECB(my_string,unknown_str,key)
cipher=cipher0
while len(cipher0)==len(cipher):
    my_string += "A"
    cipher = AES_128_ECB(my_string,unknown_str,key)
bsize = len(cipher)-len(cipher0)
print("Block size =",bsize)

Block size = 16


In [52]:
# Detect that the function is using ECB

# feed the oracle function with a string twice as long as the block size
# to ensure that repetitions would be visible regardless of the content of 
# the unknown string

my_string = 2*bsize*"A" 
cipher = AES_128_ECB(my_string,unknown_str,key)
pred = isAESECB(cipher)
print("Is the function using ECB?", pred)

Is the function using ECB? True


In [119]:
# Knowing the block size, craft an input block that is exactly 1 byte short 
# Think about what the oracle function is going to put in that last byte position.
# --> The first byte of the unknown string after encoding

my_string = (bsize-1)*"A"
cipher = AES_128_ECB(my_string,unknown_str,key)
b = cipher[:bsize]
binascii.b2a_base64(b)
len(b)

16

In [129]:
# Make a dictionary of every possible last byte by feeding different strings to the oracle
# remembering the first block of each invocation. Match the output of the one-byte-short 
# input to one of the entries in your dictionary. 

k = 1
bytedict = {}
for i in range(256):
    my_string = (bsize-k)*"A"+chr(i)
    cipher = AES_128_ECB(my_string,unknown_str,key)
    bytedic[ cipher[:bsize] ] = i
    
b0 = bytedic[b]
print(chr(b0))

R


In [187]:
# I initially made my injection string have the block lenght, but if I want to decode the full cipher I should 
# inject a string that is as long as the cipher (thus as the plaintext plu the padding, if any)

cipher = AES_128_ECB("",unknown_str,key)
cipsize = len(cipher)

plaintext = ""

for k in range(1,cipsize+1):
    my_string = (cipsize-k)*"A"
    cipher = AES_128_ECB(my_string,unknown_str,key)
    b = cipher[:cipsize]
    bytedic = {}
    for i in range(256):
        my_string = (cipsize-k)*"A"+plaintext+chr(i)
        cipher_i = AES_128_ECB(my_string,unknown_str,key)
        bytedic[ cipher_i[:cipsize] ] = i
    plaintext += chr(bytedic[b])

print(plaintext)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



In [188]:
print(unknown_str)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

